## PySpark Example Notebook

##### Downloading & Packaging python packaging for Spark drivers and workers

You can install all the packages you need and make it distributable using `conda pack`, so that it can be provided to all the spark workers.

`conda pack` can take care of packaging both the conda and pip packages. 

In [1]:
%%capture 

!conda install -y pandas pyarrow==7.0.0 conda-pack
!pip install spacy
!python -m spacy download en_core_web_sm
!conda pack -f -o base_conda_env.tar.gz

### Setting up Spark Session

In [2]:
from delta import configure_spark_with_delta_pip
import os
import pandas as pd
import pyspark
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, LongType, ArrayType
import pyspark.sql.functions as F
import spacy
from typing import List

In [3]:
maven_packages = [
    "io.delta:delta-core_2.12:1.1.0",
    "org.apache.hadoop:hadoop-aws:3.3.1",
    # "com.johnsnowlabs.nlp:spark-nlp-spark32_2.12:3.4.3", # for sparknlp
]
maven_packages = ",".join(maven_packages)

os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages "{maven_packages}" pyspark-shell'
# required when you want to use your installed packages on spark workers
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"

In [4]:
namespace = os.environ["NAMESPACE"] # usually "firstname-lastname"
notebook_name = os.environ["NOTEBOOK_NAME"] # might be helpful

In [5]:
builder = (
    pyspark.sql.SparkSession.builder.appName(f"{namespace}-spark-app")
    .config("spark.archives", "base_conda_env.tar.gz#environment") # required when you want to use your installed packages on spark workers
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.WebIdentityTokenCredentialsProvider") # Either use built in authentication for S3
    # or a custom one with specific S3 Access and Secret Keys
    # .config("spark.hadoop.fs.s3a.access.key", os.environ['AWS_S3_ACCESS_KEY']) # optional
    # .config("spark.hadoop.fs.s3a.secret.key", os.environ['AWS_S3_SECRET_KEY']) # optional
    # The section with `spark.kubernetes.executor.volumes.persistentVolumeClaim` is for
    # specifying the usage of a loca volume to enable more storage space for Disk Spilling
    # If not need, just completely remove the properties
    # you need only to modify the necessary size for the volume under `sizeLimit`
    # .config("spark.kubernetes.executor.volumes.persistentVolumeClaim.spark-local-dir-1.options.claimName", "OnDemand") # disk storage for spilling
    # .config("spark.kubernetes.executor.volumes.persistentVolumeClaim.spark-local-dir-1.options.storageClass", "ebs-csi") # disk storage for spilling
    # .config("spark.kubernetes.executor.volumes.persistentVolumeClaim.spark-local-dir-1.options.sizeLimit", "100Gi") # disk storage for spilling
    # .config("spark.kubernetes.executor.volumes.persistentVolumeClaim.spark-local-dir-1.mount.path", "/data") # disk storage for spilling
    # .config("spark.kubernetes.executor.volumes.persistentVolumeClaim.spark-local-dir-1.mount.readOnly", "false") # disk storage for spilling
    # The section with `spark.kubernetes.node.selector` is for specifying
    # what nodes to use for the executor and in which Availability Zone (AZ)
    # They need to be in the same zone
    # .config("spark.kubernetes.node.selector.topology.ebs.csi.aws.com/zone", "eu-central-1a") # node selector
    # .config("spark.kubernetes.node.selector.plural.sh/scalingGroup", "large-mem-optimized-on-demand") # node selector
    .config("spark.executor.instances", "2") # number of Executors
    .config("spark.executor.memory", "12g") # Executor memory
    .config("spark.executor.cores", "1") # Executor cores
    .config("spark.executor.pyspark.memory", "8g")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9e0af081-cce0-431b-9152-ec7e22b922fd;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 172ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	o

Spark can infer schema at runtime but it's better to provide it as it can serve as a validation for the dataset

In [6]:
schema = StructType([
    StructField("text", StringType()),
    StructField("timestamp", TimestampType()),
    StructField("url", StringType()),
])

Doc - https://spark.apache.org/docs/latest/sql-data-sources.html

In [7]:
df = spark.read.format("parquet").schema(schema).load("s3a://opengptx/dataset/en.noclean_parquet_snappy/c4-train.00000-of-07168.json.gz")

22/04/06 02:52:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [8]:
df.printSchema()

root
 |-- text: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- url: string (nullable = true)



## Working with PySpark

### Custom Operations

There are 3 different ways to do operations on the dataframes/tables:
1. Spark SQL Functions
2. PySpark UDF (User Defined Function)
3. Pandas UDF (a.k.a. Vectorized UDFs)

We will cover examples for each type.

#### Count number of words in text using Pandas UDF

User Guide - https://spark.apache.org/docs/latest/api/python/user_guide/sql/arrow_pandas.html#pandas-udfs-a-k-a-vectorized-udfs

Docs - https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.pandas_udf.html

In [9]:
@F.pandas_udf("long")
def count_words_series_pd_udf(s: pd.Series) -> pd.Series:
    return s.str.split(" ").apply(len)

In [10]:
df.withColumn("word_count_pd_udf", count_words_series_pd_udf(F.col(("text")))).show(5)

+--------------------+-------------------+--------------------+-----------------+
|                text|          timestamp|                 url|word_count_pd_udf|
+--------------------+-------------------+--------------------+-----------------+
|November 24, 2016...|2019-04-24 16:35:11|http://sevendayne...|              565|
|Beginners BBQ Cla...|2019-04-25 12:57:54|https://klyq.com/...|              232|
|Download Link pag...|2019-04-20 10:50:34|https://yyen.info...|              165|
|Restore from larg...|2019-04-21 10:07:13|https://forums.ma...|             1786|
|Jet Rashie Set - ...|2019-04-20 06:57:02|https://www.skyea...|              392|
+--------------------+-------------------+--------------------+-----------------+
only showing top 5 rows



#### Count number of words using Spark SQL Functions

Docs - https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html

In [11]:
df.withColumn("word_count_native", F.size(F.split(F.col('text'), ' '))).show(5)

+--------------------+-------------------+--------------------+-----------------+
|                text|          timestamp|                 url|word_count_native|
+--------------------+-------------------+--------------------+-----------------+
|November 24, 2016...|2019-04-24 16:35:11|http://sevendayne...|              565|
|Beginners BBQ Cla...|2019-04-25 12:57:54|https://klyq.com/...|              232|
|Download Link pag...|2019-04-20 10:50:34|https://yyen.info...|              165|
|Restore from larg...|2019-04-21 10:07:13|https://forums.ma...|             1786|
|Jet Rashie Set - ...|2019-04-20 06:57:02|https://www.skyea...|              392|
+--------------------+-------------------+--------------------+-----------------+
only showing top 5 rows



#### Remove stop words using PySpark UDF
Docs - https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.udf.html

In [12]:
en = spacy.load('en_core_web_sm')
en_stopwords = en.Defaults.stop_words

In [13]:
def remove_stopwords(text: str) -> List[str]:
    lst=[]
    for token in text.split():
        if token.lower() not in en_stopwords:
            lst.append(token)

    return lst

In [14]:
remove_stopwords_udf = F.udf(lambda z: remove_stopwords(z), ArrayType(StringType()))

In [15]:
df.withColumn("text_without_stopwords", remove_stopwords_udf(F.col("text"))).show(5)

+--------------------+-------------------+--------------------+----------------------+
|                text|          timestamp|                 url|text_without_stopwords|
+--------------------+-------------------+--------------------+----------------------+
|November 24, 2016...|2019-04-24 16:35:11|http://sevendayne...|  [November, 24,, 2...|
|Beginners BBQ Cla...|2019-04-25 12:57:54|https://klyq.com/...|  [Beginners, BBQ, ...|
|Download Link pag...|2019-04-20 10:50:34|https://yyen.info...|  [Download, Link, ...|
|Restore from larg...|2019-04-21 10:07:13|https://forums.ma...|  [Restore, larger,...|
|Jet Rashie Set - ...|2019-04-20 06:57:02|https://www.skyea...|  [Jet, Rashie, Set...|
+--------------------+-------------------+--------------------+----------------------+
only showing top 5 rows



### Example
Let's add UUID to our original table and create a metadata table, with count of words and stopwords in text.
We will later join the metadata table with our original table.

#### Add UUID to Dataframe

In [16]:
df = df.withColumn("uuid", F.expr("uuid()"))

In [17]:
# reorder the columns
df = df.select("uuid", "text", "timestamp", "url")

In [18]:
df.show(5)

+--------------------+--------------------+-------------------+--------------------+
|                uuid|                text|          timestamp|                 url|
+--------------------+--------------------+-------------------+--------------------+
|76819797-942a-405...|November 24, 2016...|2019-04-24 16:35:11|http://sevendayne...|
|e8cf0316-8eba-404...|Beginners BBQ Cla...|2019-04-25 12:57:54|https://klyq.com/...|
|6fa41fc7-779a-42c...|Download Link pag...|2019-04-20 10:50:34|https://yyen.info...|
|089d39bd-4ce7-492...|Restore from larg...|2019-04-21 10:07:13|https://forums.ma...|
|75a78088-ad0f-477...|Jet Rashie Set - ...|2019-04-20 06:57:02|https://www.skyea...|
+--------------------+--------------------+-------------------+--------------------+
only showing top 5 rows



#### Create Metadata table

In [19]:
@F.udf(returnType=LongType())
def count_stopwords(text: str) -> int:
    return len(text.split()) - len(remove_stopwords(text))

In [20]:
# create metadata table
df_metadata = df.select("uuid", count_words_series_pd_udf(df.text).alias("count_words"), count_stopwords(df.text).alias("count_stopwords"))

In [21]:
df_metadata.show(5)

+--------------------+-----------+---------------+
|                uuid|count_words|count_stopwords|
+--------------------+-----------+---------------+
|76819797-942a-405...|        565|            197|
|e8cf0316-8eba-404...|        232|             99|
|6fa41fc7-779a-42c...|        165|             33|
|089d39bd-4ce7-492...|       1786|            460|
|75a78088-ad0f-477...|        392|            123|
+--------------------+-----------+---------------+
only showing top 5 rows



#### Dataframe Joins
Docs - https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.join.html

In [22]:
df_full = df.join(df_metadata, on="uuid", how="left")

In [23]:
# Joins require reshuffling of data, thus the order mismatch
df_full.show(5)

+--------------------+--------------------+-------------------+--------------------+-----------+---------------+
|                uuid|                text|          timestamp|                 url|count_words|count_stopwords|
+--------------------+--------------------+-------------------+--------------------+-----------+---------------+
|089d39bd-4ce7-492...|Restore from larg...|2019-04-21 10:07:13|https://forums.ma...|       1786|            460|
|6fa41fc7-779a-42c...|Download Link pag...|2019-04-20 10:50:34|https://yyen.info...|        165|             33|
|e8cf0316-8eba-404...|Beginners BBQ Cla...|2019-04-25 12:57:54|https://klyq.com/...|        232|             99|
|38b3621e-cc99-4af...|Skip to Main Cont...|2019-04-23 14:13:51|https://www.expre...|        466|             93|
|75a78088-ad0f-477...|Jet Rashie Set - ...|2019-04-20 06:57:02|https://www.skyea...|        392|            123|
+--------------------+--------------------+-------------------+--------------------+-----------+

#### Saving Dataframe

In [24]:
(
    df_full.write.format("parquet")
    .option("compression", "snappy")
    .mode("overwrite")
    .save("s3a://opengptx/dataset/demo/c4-train.00000-of-07168_full.parquet")
)

### Further Reading

Working with DataFrame - https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html

Pandas API on Spark - https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html
